# Ingestion API Transaction Data Generation Script

### Import Packages

In [14]:
from faker import Faker
import random
import datetime
import pandas as pd

### Initialize Faker

In [15]:
fake = Faker()

### Generate List of Account IDs we want to create transactions for

In [16]:
account_ids = ["001al00000EkcjqAAB"]

### Generate Baseline Variable Definitions

In [18]:
category_mcc_mapping = {
    "Auto & Transport": {"mcc_range": (4000, 4799), "description": "Transportation services for auto and transport transactions"},
    "Bills & Utilities": {"mcc_range": (4800, 4999), "description": "Utility services for bills and utilities"},
    "Business Services": {"mcc_range": (7300, 7999), "description": "Business services"},
    "Education": {"mcc_range": (8000, 8999), "description": "Professional services and membership organizations for education"},
    "Entertainment": {"mcc_range": (7300, 7999), "description": "Entertainment activities"},
    "Fees & Charges": {"mcc_range": (7300, 7999), "description": "Fees and charges"},
    "Financial": {"mcc_range": (7300, 7999), "description": "Financial and banking services"},
    "Food & Dining": {"mcc_range": (5700, 7299), "description": "Food and restaurant services"},
    "Gifts & Donations": {"mcc_range": (5700, 7299), "description": "Gifts and donations"},
    "Health & Fitness": {"mcc_range": (1500, 2999), "description": "Health and fitness services such as gyms, classes, etc."},
    "Home": {"mcc_range": (7300, 7999), "description": "Home improvement stores and home-related services"},
    "Investments": {"mcc_range": (7300, 7999), "description": "Investments and financial services"},
    "Kids": {"mcc_range": (1500, 2999), "description": "Childcare or child-related services"},
    "Loans": {"mcc_range": (7300, 7999), "description": "Loans and loan payments"},
    "Personal Care": {"mcc_range": (1500, 2999), "description": "Personal care items and services"},
    "Pets": {"mcc_range": (7300, 7999), "description": "Pet stores, pet care, and pet grooming services"},
    "Shopping": {"mcc_range": (5000, 5599), "description": "Retail stores"},
    "Taxes": {"mcc_range": (9000, 9999), "description": "Taxes and internal revenue services"},
    "Transfer": {"mcc_range": (7300, 7999), "description": "Financial money transfers"},
    "Travel": {"mcc_range": (4000, 4799), "description": "Travel services such as airlines, hotels, trains, auto rentals, etc."},
    "Paycheck": {"mcc_range": (7300, 7999), "description": "Direct deposit"},
    "Bonus": {"mcc_range": (7300, 7999), "description": "Performance or annual bonuses or commissions"},
    "Rental Income": {"mcc_range": (7300, 7999), "description": "Rental income"},
    "Interest Income": {"mcc_range": (7300, 7999), "description": "Account interest accrued"},
    "Reimbursement": {"mcc_range": (7300, 7999), "description": "Purchase or service reimbursement"},
    "Returned Purchase": {"mcc_range": (5000, 5599), "description": "Returns of merchandise"},
    "Other Income": {"mcc_range": (5700, 7299), "description": "Miscellaneous purchases"}
}

In [19]:
# MCC categories
mcc_categories = {
    (1500, 2999): "Contracted services",
    (4000, 4799): "Transportation services",
    (4800, 4999): "Utility services",
    (5000, 5599): "Retail outlet services",
    (5700, 7299): "Miscellaneous stores",
    (7300, 7999): "Business services",
    (8000, 8999): "Professional services",
    (9000, 9999): "Government services"
}

In [20]:
# Transaction statuses
transaction_statuses = ["Pending", "Posted", "Posted", "Posted"]  # Posted is more common

In [21]:
# Transaction subtypes
transaction_subtypes = {
    "Auto & Transport": ["Auto Deposit", "Transport Payment", "Auto Purchase"],
    "Bills & Utilities": ["Utility Bill Payment", "Bills Payment", "Utility Deposit"],
    "Business Services": ["Business Fee", "Service Charge", "Professional Membership"],
    "Education": ["Tuition Payment", "Education Expense", "School Fee"],
    "Entertainment": ["Entertainment Expense", "Event Ticket Purchase", "Movie Rental"],
    "Fees & Charges": ["Service Fee", "Charge Fee", "Transaction Fee"],
    "Financial": ["Bank Fee", "Financial Charge", "Financial Service"],
    "Food & Dining": ["Restaurant Bill Payment", "Food Purchase", "Dining Expense"],
    "Gifts & Donations": ["Gift Purchase", "Donation", "Charity Payment"],
    "Health & Fitness": ["Gym Membership", "Fitness Expense", "Healthcare Payment"],
    "Home": ["Home Improvement Purchase", "Home Service Payment", "Home Maintenance"],
    "Investments": ["Investment Fee", "Investment Expense", "Financial Investment"],
    "Kids": ["Childcare Payment", "Kid Purchase", "Child Expense"],
    "Loans": ["Loan Payment", "Loan Expense", "Loan Fee"],
    "Personal Care": ["Personal Care Purchase", "Personal Service Payment", "Care Expense"],
    "Pets": ["Pet Purchase", "Pet Care Payment", "Pet Expense"],
    "Shopping": ["Retail Purchase", "Shopping Expense", "Store Payment"],
    "Taxes": ["Tax Payment", "Tax Expense", "Tax Service"],
    "Transfer": ["Transfer Service", "Money Transfer", "Financial Transfer"],
    "Travel": ["Travel Expense", "Travel Ticket Purchase", "Travel Payment"],
    "Paycheck": ["Direct Deposit", "Salary Payment", "Wage Deposit"],
    "Bonus": ["Bonus Payment", "Performance Bonus", "Commission Payment"],
    "Rental Income": ["Rental Payment", "Rental Deposit", "Rental Income"],
    "Interest Income": ["Interest Earned", "Interest Payment", "Interest Deposit"],
    "Reimbursement": ["Reimbursement", "Expense Reimbursement", "Service Refund"],
    "Returned Purchase": ["Return Refund", "Product Return", "Refund Payment"],
    "Other Income": ["Miscellaneous Payment", "Misc. Deposit", "Misc. Income"]
}

### Generate Function to Create Transactions

In [26]:
def generate_transaction():
    transaction_category = random.choice(list(category_mcc_mapping.keys()))
    mcc_range = category_mcc_mapping[transaction_category]["mcc_range"]
    mcc = random.randint(mcc_range[0], mcc_range[1])
    mcc_description = mcc_categories[(mcc_range[0], mcc_range[1])]

    transaction_date = fake.date_time_between(start_date="-5d", end_date="now").strftime("%Y-%m-%d %H:%M:%S")

    transaction_subtype_options = transaction_subtypes[transaction_category]
    transaction_subtype = random.choice(transaction_subtype_options)

    if any(word in transaction_subtype.lower() for word in ["deposit", "interest earned", "reversal", "misc. credit"]):
        transaction_type = "Credit"
    else:
        transaction_type = "Debit"
    
    transaction = {
        "accountid": random.choice(account_ids),
        "transactionid": fake.uuid4()[:100],
        "transaction_date": transaction_date,
        "amount": round(random.uniform(1, 2000), 2),
        "description": category_mcc_mapping[transaction_category]["description"],
        "transaction_category": transaction_category,
        "mcc": mcc,
        "mcc_description": mcc_description,
        "transaction_status": random.choice(transaction_statuses),
        "post_date": (datetime.datetime.strptime(transaction_date, "%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S"),
        "currency": "USD",
        "is_disputed": random.choices([0, 1], weights=[0.99, 0.01], k=1)[0],
        "transaction_type": transaction_type,
        "transaction_subtype": transaction_subtype,
        "running_balance": round(random.uniform(1000, 100000), 2),
        "source_transaction_type": transaction_type,
        "transaction_name": fake.uuid4(),
        "data_date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "is_deleted": False
    }
    return transaction

### Variable for number of transactions we want to generate

In [27]:
n_transactions = 10

### Execute Generation

In [28]:
transactions = [generate_transaction() for _ in range(n_transactions)]

### Save Output to a Dataframe

In [29]:
df = pd.DataFrame(transactions)
print(df)

            accountid                         transactionid  \
0  001al00000EkcjqAAB  14ae7062-82ab-44d0-bf1c-7a5f78af35ac   
1  001al00000EkcjqAAB  c93b1b53-83ac-43c6-81d4-7fb31df030a4   
2  001al00000EkcjqAAB  1441175d-6d7d-4733-8836-d004c6af0301   
3  001al00000EkcjqAAB  6210b11a-9f01-45f0-8253-a64f0eec381f   
4  001al00000EkcjqAAB  c7de44e3-9e21-4a3e-9ebf-af361eebc3d9   
5  001al00000EkcjqAAB  6d5a32d1-aed6-4bd8-bb32-8471968b9f36   
6  001al00000EkcjqAAB  14878192-650d-46e5-be0d-1543cc52434b   
7  001al00000EkcjqAAB  445b54ba-e462-44e8-9976-9b527701b60d   
8  001al00000EkcjqAAB  0192a237-fe67-4d6f-b53d-750d33e6d546   
9  001al00000EkcjqAAB  4538f6c9-c561-4fa5-9694-92a31567cdb2   

      transaction_date  amount                                   description  \
0  2024-02-05 16:14:07  592.56           Childcare or child-related services   
1  2024-16-05 05:04:48  601.98  Performance or annual bonuses or commissions   
2  2024-23-04 01:19:34  583.14            Investments and financia

### Output Transactions to a CSV File (Batch)

In [ ]:
output = df.to_csv('/Users/jsifontes/Documents/Dev/Data Science/DataSets/Data Cloud/Transactions2.csv', index=False)

### Output Trasactions to a JSON file (Streaming)

In [ ]:
outputjson = df.to_json(orient="records")
data_envelope = f'{{"data": {outputjson}}}'
print(data_envelope)

### Write Output to JSON File

In [ ]:
with open('/Users/jsifontes/Documents/Dev/Data Cloud/Datasets/Transactions.json','w') as output:
    output.write(data_envelope)